In [ ]:
import pandas as pd
import os
from tqdm import tqdm

In [ ]:
os.mkdir('separate_spectogram')

In [ ]:
train = pd.read_csv('/kaggle/input/hms-harmful-brain-activity-classification/train.csv')

In [ ]:
k = 0
for eeg in tqdm(train['eeg_id'].unique()):
    df = train[train['eeg_id'] == eeg][['spectrogram_label_offset_seconds', 'expert_consensus', 'spectrogram_id']]
    specto_id = df['spectrogram_id'].values[0]
    specto = pd.read_parquet(f'/kaggle/input/hms-harmful-brain-activity-classification/train_spectrograms/{specto_id}.parquet')
    # Initialize variables for tracking class and times
    current_class = None
    start_time = None
    end_time = None

    for row in df.itertuples():
        if row.expert_consensus != current_class:
            if current_class is not None:
                start_time = int(start_time)
                if end_time:
                    end_time = int(end_time)
                else:
                    end_time = specto.shape[0]
                k += 1
                specto.loc[start_time:end_time,:].to_parquet(f"separate_spectogram/{eeg}-{specto_id}_{current_class}.parquet")
            
            current_class = row.expert_consensus
            start_time = row.spectrogram_label_offset_seconds
        else:
            # Update end time for the current class
            end_time = row.spectrogram_label_offset_seconds

    # Print the times for the last class
    start_time = int(start_time)
    if end_time:
        end_time = int(end_time)
    else:
        end_time = specto.shape[0]
    k += 1
    specto.loc[start_time:end_time,:].to_parquet(f"separate_spectogram/{eeg}-{specto_id}_{current_class}.parquet")


In [ ]:
import os

len(os.listdir('/kaggle/working/separate_spectogram/'))